# This code cleans the Divvy Trip data 

It checks for duplicates and removes trips under 5 minutes or trips over 1 day. 5 minutes is used as an estimate for a 2 mile trip. 
It then exports the remaining data to a csv. 

In [16]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import re

In [17]:
bikes = pd.read_csv('data/Chicago/bikes/202204-divvy-tripdata.csv') #reads divvy trip information as bikes

In [18]:
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,3564070EEFD12711,electric_bike,2022-04-06 17:42:48,2022-04-06 17:54:36,Paulina St & Howard St,515,University Library (NU),605,42.019135,-87.673532,42.052939,-87.673447,member
1,0B820C7FCF22F489,classic_bike,2022-04-24 19:23:07,2022-04-24 19:43:17,Wentworth Ave & Cermak Rd,13075,Green St & Madison St,TA1307000120,41.853085,-87.631931,41.881892,-87.648789,member
2,89EEEE32293F07FF,classic_bike,2022-04-20 19:29:08,2022-04-20 19:35:16,Halsted St & Polk St,TA1307000121,Green St & Madison St,TA1307000120,41.871840,-87.646640,41.881892,-87.648789,member
3,84D4751AEB31888D,classic_bike,2022-04-22 21:14:06,2022-04-22 21:23:29,Wentworth Ave & Cermak Rd,13075,Delano Ct & Roosevelt Rd,KA1706005007,41.853085,-87.631931,41.867491,-87.632190,casual
4,5664BCF0D1DE7A8B,electric_bike,2022-04-16 15:56:30,2022-04-16 16:02:11,Halsted St & Polk St,TA1307000121,Clinton St & Madison St,TA1305000032,41.871808,-87.646574,41.882242,-87.641066,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371244,C5A123D7BF8D350A,electric_bike,2022-04-22 15:54:11,2022-04-22 16:20:59,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892296,-87.612198,41.910475,-87.696894,member
371245,F7FCC7C26D8D137D,electric_bike,2022-04-21 20:18:17,2022-04-21 20:46:45,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892295,-87.612323,41.910475,-87.696894,member
371246,43D351300A40000A,classic_bike,2022-04-21 16:46:02,2022-04-21 17:15:05,Franklin St & Monroe St,TA1309000007,St. Clair St & Erie St,13016,41.880317,-87.635185,41.894345,-87.622798,member
371247,1618BFEEA7B566EF,electric_bike,2022-04-16 13:19:44,2022-04-16 13:37:31,Ashland Ave & Blackhawk St,13224,Southport Ave & Waveland Ave,13235,41.907094,-87.667217,41.948150,-87.663940,casual


In [19]:
bikes.shape

(371249, 13)

In [20]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371249 entries, 0 to 371248
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             371249 non-null  object 
 1   rideable_type       371249 non-null  object 
 2   started_at          371249 non-null  object 
 3   ended_at            371249 non-null  object 
 4   start_station_name  300362 non-null  object 
 5   start_station_id    300362 non-null  object 
 6   end_station_name    295961 non-null  object 
 7   end_station_id      295961 non-null  object 
 8   start_lat           371249 non-null  float64
 9   start_lng           371249 non-null  float64
 10  end_lat             370932 non-null  float64
 11  end_lng             370932 non-null  float64
 12  member_casual       371249 non-null  object 
dtypes: float64(4), object(9)
memory usage: 36.8+ MB


In [21]:
bikes.describe()

,start_lat,start_lng,end_lat,end_lng
count,371249.000000,371249.000000,370932.000000,370932.000000
mean,41.896281,-87.647990,41.896565,-87.648172
std,0.050880,0.033296,0.051021,0.033399
min,41.648501,-87.833254,41.630000,-87.850000
25%,41.879255,-87.663940,41.879344,-87.664020
50%,41.895978,-87.643340,41.896373,-87.643749
75%,41.928773,-87.627834,41.929143,-87.627834
max,42.070000,-87.520000,42.080000,-87.520000


In [22]:
test = bikes.duplicated(subset=['ride_id'], keep='first') 
test #tests to see if there are duplicates (where bike id, start time, and stop time are the same)

0         False
1         False
2         False
3         False
4         False
          ...  
371244    False
371245    False
371246    False
371247    False
371248    False
Length: 371249, dtype: bool

In [23]:
test.value_counts() #shows that there are no duplicates

False    371249
dtype: int64

In [24]:
bikes['start_dt'] = pd.to_datetime(bikes['started_at'].astype(str)) #creates datetime columns to calculate duration
bikes['end_dt'] = pd.to_datetime(bikes['ended_at'].astype(str))

bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_dt,end_dt
0,3564070EEFD12711,electric_bike,2022-04-06 17:42:48,2022-04-06 17:54:36,Paulina St & Howard St,515,University Library (NU),605,42.019135,-87.673532,42.052939,-87.673447,member,2022-04-06 17:42:48,2022-04-06 17:54:36
1,0B820C7FCF22F489,classic_bike,2022-04-24 19:23:07,2022-04-24 19:43:17,Wentworth Ave & Cermak Rd,13075,Green St & Madison St,TA1307000120,41.853085,-87.631931,41.881892,-87.648789,member,2022-04-24 19:23:07,2022-04-24 19:43:17
2,89EEEE32293F07FF,classic_bike,2022-04-20 19:29:08,2022-04-20 19:35:16,Halsted St & Polk St,TA1307000121,Green St & Madison St,TA1307000120,41.871840,-87.646640,41.881892,-87.648789,member,2022-04-20 19:29:08,2022-04-20 19:35:16
3,84D4751AEB31888D,classic_bike,2022-04-22 21:14:06,2022-04-22 21:23:29,Wentworth Ave & Cermak Rd,13075,Delano Ct & Roosevelt Rd,KA1706005007,41.853085,-87.631931,41.867491,-87.632190,casual,2022-04-22 21:14:06,2022-04-22 21:23:29
4,5664BCF0D1DE7A8B,electric_bike,2022-04-16 15:56:30,2022-04-16 16:02:11,Halsted St & Polk St,TA1307000121,Clinton St & Madison St,TA1305000032,41.871808,-87.646574,41.882242,-87.641066,member,2022-04-16 15:56:30,2022-04-16 16:02:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371244,C5A123D7BF8D350A,electric_bike,2022-04-22 15:54:11,2022-04-22 16:20:59,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892296,-87.612198,41.910475,-87.696894,member,2022-04-22 15:54:11,2022-04-22 16:20:59
371245,F7FCC7C26D8D137D,electric_bike,2022-04-21 20:18:17,2022-04-21 20:46:45,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892295,-87.612323,41.910475,-87.696894,member,2022-04-21 20:18:17,2022-04-21 20:46:45
371246,43D351300A40000A,classic_bike,2022-04-21 16:46:02,2022-04-21 17:15:05,Franklin St & Monroe St,TA1309000007,St. Clair St & Erie St,13016,41.880317,-87.635185,41.894345,-87.622798,member,2022-04-21 16:46:02,2022-04-21 17:15:05
371247,1618BFEEA7B566EF,electric_bike,2022-04-16 13:19:44,2022-04-16 13:37:31,Ashland Ave & Blackhawk St,13224,Southport Ave & Waveland Ave,13235,41.907094,-87.667217,41.948150,-87.663940,casual,2022-04-16 13:19:44,2022-04-16 13:37:31


In [25]:
bikes['duration'] = (bikes.end_dt-bikes.start_dt).dt.total_seconds() #calulates duration 
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_dt,end_dt,duration
0,3564070EEFD12711,electric_bike,2022-04-06 17:42:48,2022-04-06 17:54:36,Paulina St & Howard St,515,University Library (NU),605,42.019135,-87.673532,42.052939,-87.673447,member,2022-04-06 17:42:48,2022-04-06 17:54:36,708.0
1,0B820C7FCF22F489,classic_bike,2022-04-24 19:23:07,2022-04-24 19:43:17,Wentworth Ave & Cermak Rd,13075,Green St & Madison St,TA1307000120,41.853085,-87.631931,41.881892,-87.648789,member,2022-04-24 19:23:07,2022-04-24 19:43:17,1210.0
2,89EEEE32293F07FF,classic_bike,2022-04-20 19:29:08,2022-04-20 19:35:16,Halsted St & Polk St,TA1307000121,Green St & Madison St,TA1307000120,41.871840,-87.646640,41.881892,-87.648789,member,2022-04-20 19:29:08,2022-04-20 19:35:16,368.0
3,84D4751AEB31888D,classic_bike,2022-04-22 21:14:06,2022-04-22 21:23:29,Wentworth Ave & Cermak Rd,13075,Delano Ct & Roosevelt Rd,KA1706005007,41.853085,-87.631931,41.867491,-87.632190,casual,2022-04-22 21:14:06,2022-04-22 21:23:29,563.0
4,5664BCF0D1DE7A8B,electric_bike,2022-04-16 15:56:30,2022-04-16 16:02:11,Halsted St & Polk St,TA1307000121,Clinton St & Madison St,TA1305000032,41.871808,-87.646574,41.882242,-87.641066,member,2022-04-16 15:56:30,2022-04-16 16:02:11,341.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371244,C5A123D7BF8D350A,electric_bike,2022-04-22 15:54:11,2022-04-22 16:20:59,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892296,-87.612198,41.910475,-87.696894,member,2022-04-22 15:54:11,2022-04-22 16:20:59,1608.0
371245,F7FCC7C26D8D137D,electric_bike,2022-04-21 20:18:17,2022-04-21 20:46:45,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892295,-87.612323,41.910475,-87.696894,member,2022-04-21 20:18:17,2022-04-21 20:46:45,1708.0
371246,43D351300A40000A,classic_bike,2022-04-21 16:46:02,2022-04-21 17:15:05,Franklin St & Monroe St,TA1309000007,St. Clair St & Erie St,13016,41.880317,-87.635185,41.894345,-87.622798,member,2022-04-21 16:46:02,2022-04-21 17:15:05,1743.0
371247,1618BFEEA7B566EF,electric_bike,2022-04-16 13:19:44,2022-04-16 13:37:31,Ashland Ave & Blackhawk St,13224,Southport Ave & Waveland Ave,13235,41.907094,-87.667217,41.948150,-87.663940,casual,2022-04-16 13:19:44,2022-04-16 13:37:31,1067.0


In [26]:
#filters out trips shorter than 5 minutes and longer than 1 day
bikes_clean = bikes.loc[(bikes['duration'] >= 300) & (bikes['duration'] <= 86400)] 
bikes_clean.shape

(285760, 16)

In [27]:
bikes_clean

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_dt,end_dt,duration
0,3564070EEFD12711,electric_bike,2022-04-06 17:42:48,2022-04-06 17:54:36,Paulina St & Howard St,515,University Library (NU),605,42.019135,-87.673532,42.052939,-87.673447,member,2022-04-06 17:42:48,2022-04-06 17:54:36,708.0
1,0B820C7FCF22F489,classic_bike,2022-04-24 19:23:07,2022-04-24 19:43:17,Wentworth Ave & Cermak Rd,13075,Green St & Madison St,TA1307000120,41.853085,-87.631931,41.881892,-87.648789,member,2022-04-24 19:23:07,2022-04-24 19:43:17,1210.0
2,89EEEE32293F07FF,classic_bike,2022-04-20 19:29:08,2022-04-20 19:35:16,Halsted St & Polk St,TA1307000121,Green St & Madison St,TA1307000120,41.871840,-87.646640,41.881892,-87.648789,member,2022-04-20 19:29:08,2022-04-20 19:35:16,368.0
3,84D4751AEB31888D,classic_bike,2022-04-22 21:14:06,2022-04-22 21:23:29,Wentworth Ave & Cermak Rd,13075,Delano Ct & Roosevelt Rd,KA1706005007,41.853085,-87.631931,41.867491,-87.632190,casual,2022-04-22 21:14:06,2022-04-22 21:23:29,563.0
4,5664BCF0D1DE7A8B,electric_bike,2022-04-16 15:56:30,2022-04-16 16:02:11,Halsted St & Polk St,TA1307000121,Clinton St & Madison St,TA1305000032,41.871808,-87.646574,41.882242,-87.641066,member,2022-04-16 15:56:30,2022-04-16 16:02:11,341.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371244,C5A123D7BF8D350A,electric_bike,2022-04-22 15:54:11,2022-04-22 16:20:59,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892296,-87.612198,41.910475,-87.696894,member,2022-04-22 15:54:11,2022-04-22 16:20:59,1608.0
371245,F7FCC7C26D8D137D,electric_bike,2022-04-21 20:18:17,2022-04-21 20:46:45,Streeter Dr & Grand Ave,13022,California Ave & North Ave,13258,41.892295,-87.612323,41.910475,-87.696894,member,2022-04-21 20:18:17,2022-04-21 20:46:45,1708.0
371246,43D351300A40000A,classic_bike,2022-04-21 16:46:02,2022-04-21 17:15:05,Franklin St & Monroe St,TA1309000007,St. Clair St & Erie St,13016,41.880317,-87.635185,41.894345,-87.622798,member,2022-04-21 16:46:02,2022-04-21 17:15:05,1743.0
371247,1618BFEEA7B566EF,electric_bike,2022-04-16 13:19:44,2022-04-16 13:37:31,Ashland Ave & Blackhawk St,13224,Southport Ave & Waveland Ave,13235,41.907094,-87.667217,41.948150,-87.663940,casual,2022-04-16 13:19:44,2022-04-16 13:37:31,1067.0


In [13]:
bikes_clean.to_csv('data/Chicago/cleaned/bikes/bikes_clean_m12.csv', index=False)
#exports clean dataset to csv

In [14]:
bikes['start_dt'].min()

Timestamp('2022-12-01 00:01:22')

In [15]:
bikes['end_dt'].max()

Timestamp('2023-01-02 04:56:45')